In [19]:
import torch
from torch import nn
from torch.autograd import Variable
import torch.optim as optim
from collections import OrderedDict
from scipy.stats import ortho_group
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from time import time
%matplotlib inline

### Setup

In [20]:
# use nvidia gpu if available
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
# only float32 tested
dtype = torch.float32
    
print("Device:", device)
print("Data type:", dtype)

Device: cuda
Data type: torch.float32


### Define Encoder/Decoder Logic

In [21]:
class L2Norm(nn.Module):
    """L2Norm model class -- a layer class for normalizing a matrix of vectors
    
    Args:
        dim (int) - which dimension to normalize along. 0 for column vectors, 1 for row vectors
    """
    def __init__(self, dim=1):
        super(L2Norm, self).__init__()
        
        self.dim=dim
        
    def forward(self,input):
        return nn.functional.normalize(input, p=2, dim=self.dim)

In [45]:
class Hamiltonian(nn.Module):
    """Hamiltonian -- a class used to help learn physically relevant sub-manifold
    
    Args:
        H - the hamiltonian matrix in the original basis
        N (int) - the size of the original basis
        N_proj (int) - the size of the projected basis
    """
    
    def __init__(self, H, N, N_proj):

        super(Hamiltonian, self).__init__()
        
        # initialize the class variables
        self.H = H
        self.N = N
        self.N_proj = N_proj
        self.eye = torch.eye(N_proj, device=device, dtype=dtype)
        
        # initialize the encoder and decoder architectures
        self.decoder = nn.Sequential(OrderedDict([('matmul', nn.Linear(self.N_proj, self.N, bias=False)),
                                                  ('normalize', L2Norm())
                                                  ]))

    # return the current cost matrix
    def forward(self):
        M = self.decoder(self.eye)
        return M @ self.H @ M.t(), M @ M.t()

In [46]:
class Cost():
    """Cost class -- used to calculate goodness of projection map for optimization purposes
    
    Args:
        N_proj - number of basis states in the projected space
        alpha (float) - the weight of the orthogonality cost relative to the energy cost (units of energy)
    """
    
    def __init__(self, N_proj, alpha):
        super(Cost, self).__init__()
        
        self.N_proj = N_proj
        self.eye = torch.eye(N_proj, device=device, dtype=dtype)
        self.sorter = torch.tensor([[1/(i+1)/(j+1) for i in range(self.N_proj)] for j in range(self.N_proj)], device=device, dtype=dtype)
        
    def __call__(self, Hproj, Iproj):
        cost_matrix = (Hproj*self.sorter)**2 + alpha * (Iproj - self.eye)**2
        return cost_matrix.sum()/self.N_proj**2

### Synthesize a reasonable test Hamiltonian

In [47]:
def generate_H(N, N_proj):
    eigs,_ = (torch.rand(N, device=device, dtype=dtype)*N).sort()
    Hgen_proj = torch.diag(eigs)
    Wgen = torch.tensor(ortho_group.rvs(N), device=device, dtype=dtype)
    
    Hgen = Wgen @ Hgen_proj @ Wgen.t()
    
    return Hgen, Wgen, eigs

### Optimization

In [43]:
def early_stop(errs, rel_tol, patience=2):
    stop=True
    for i in range(patience):
        rel_change = np.abs((errs[-(i+1)] - errs[-(i+2)]) / errs[-(i+1)])
        stop = stop and rel_change < rel_tol
    return stop

def optimize(n_iter, n_save, show_progress=True, stop_early=True, rel_tol=0.01, patience=2):
    model.decoder.matmul.reset_parameters()
    running_loss= 0
    start_time = time()
    
    # store progress every n_save iters
    j=0
    its = []
    errs = []
    Hps = []
    Ips = []
    ts = []
    
    # iterate
    for i in range(n_iter):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        Hp, Ip = model()
        loss = criterion(Hp, Ip)
        loss.backward()
        optimizer.step()

        # print /save statistics
        running_loss += loss.item()
        if i % n_save == 0:
            # save progress
            its.append(i)
            errs.append(running_loss)
            Hps.append(Hp)
            Ips.append(Ip)
            ts.append(time() - start_time)
            
            # print progress 
            if show_progress:
                change = errs[j]-errs[j-1] if j>0 else 0
                print('[%d] loss: %.8f. diff: %.8f. time: %.4f' % (i + 1, running_loss / 10, change, ts[-1]))
            
            if j>=patience and stop_early and early_stop(errs, rel_tol, patience):
                print("Early stopping criteria met.")
                break
            
            running_loss = 0.0
            j+=1
            

    print('Finished Training')
    return its, errs, Hps, Ips, ts

### Time Complexity Measurements

#### Scaling w/ N

In [19]:
# use cpu for timing due to massive gpu parallelization
device = torch.device('cpu')

In [20]:
Ns = np.logspace(2,3.2,5).astype(int)
N_proj = 10
repeats = 10
lam=1
max_iter = 10000
n_print = 200
criterion = Cost(N_proj, alpha)

In [ ]:
average_time = []
stddev_time = []
for N in Ns:
    t = []
    for r in range(repeats):
        print("N=%i. Nproj=%i. repeat=%i" % (N, N_proj, r+1))
        H, _, _ = generate_H(N, N_proj)
        model = Hamiltonian(H, N, N_proj).to(device=device)
        optimizer = optim.Rprop(model.parameters(), lr=0.1)

        _, _, _, _, ts = optimize(max_iter, n_print, show_progress=False, stop_early=True, rel_tol=0.01, patience=3)

        t.append(ts[-1])
        
    average_time.append(np.average(t))
    stddev_time.append(np.std(t))

N=100. Nproj=10. repeat=1
Early stopping criteria met.
Finished Training
N=100. Nproj=10. repeat=2
Early stopping criteria met.
Finished Training
N=100. Nproj=10. repeat=3
Early stopping criteria met.
Finished Training
N=100. Nproj=10. repeat=4
Early stopping criteria met.
Finished Training
N=100. Nproj=10. repeat=5
Early stopping criteria met.
Finished Training
N=100. Nproj=10. repeat=6
Early stopping criteria met.
Finished Training
N=100. Nproj=10. repeat=7
Early stopping criteria met.
Finished Training
N=100. Nproj=10. repeat=8
Early stopping criteria met.
Finished Training
N=100. Nproj=10. repeat=9
Early stopping criteria met.
Finished Training
N=100. Nproj=10. repeat=10
Early stopping criteria met.
Finished Training
N=199. Nproj=10. repeat=1
Early stopping criteria met.
Finished Training
N=199. Nproj=10. repeat=2
Early stopping criteria met.
Finished Training
N=199. Nproj=10. repeat=3
Early stopping criteria met.
Finished Training
N=199. Nproj=10. repeat=4
Early stopping criteria 

In [ ]:
plt.loglog(Ns, average_time, 'bo-')
plt.grid()

In [ ]:
np.polyfit(np.log(Ns), np.log(average_time), 1)

$t \sim \mathcal{O}\left(N^{0.83}\right) $